<a href="https://colab.research.google.com/github/tylerwalkerbrown/stock_analysis/blob/main/News_Setiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes
Change Runtime to GPU in Runtime Settings 

## Packages needed for twitter 

In [ ]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall
!pip install tensorflow==2.6.0
!pip install tf-models-official
!pip install --upgrade tensorflow
!pip install --upgrade numpy pandas matplotlib seaborn nltk scikit-learn wordcloud

In [ ]:
#import os

# Specify the folder path
#folder_path = '/content/drive/MyDrive/stocks_app'

# Create the folder if it doesn't exist
#if not os.path.exists(folder_path):
 #   os.makedirs(folder_path)

# Verify if the folder was created
#if os.path.exists(folder_path):
 #   print("Folder created successfully.")
#else:
  #  print("Failed to create the folder.")


Folder created successfully.


## Packages Need for Deeplearning

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from wordcloud import WordCloud

In [ ]:
# Load the data
df = pd.read_csv("/content/drive/MyDrive/all-data.csv", encoding='latin-1')

y = df.iloc[:, 0].values
x = df.iloc[:, 1].values

# Convert string labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Tokenize the text and convert it into sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to have the same length
max_sequence_length = max(len(seq) for seq in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Define the neural network architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(units=128))
model.add(Dense(units=3, activation="softmax"))  # Use 3 units for multiclass classification

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


Epoch 1/5
122/122 [==============================] - 18s 128ms/step - loss: 0.8321 - accuracy: 0.6409
Epoch 2/5
122/122 [==============================] - 5s 39ms/step - loss: 0.5322 - accuracy: 0.7761
Epoch 3/5
122/122 [==============================] - 3s 21ms/step - loss: 0.2714 - accuracy: 0.9030
Epoch 4/5
122/122 [==============================] - 2s 20ms/step - loss: 0.1218 - accuracy: 0.9582
Epoch 5/5
31/31 [==============================] - 1s 6ms/step - loss: 1.1824 - accuracy: 0.7162
Test Loss: 1.1824408769607544
Test Accuracy: 0.7162022590637207


In [ ]:
import os
from tensorflow.keras.models import save_model

# Define the folder path
folder_path = "/content/drive/MyDrive/stocks_app"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the model in the folder
model_path = os.path.join(folder_path, "trained_model.h5")
save_model(model, model_path)

# Save other files in the folder
file_path = os.path.join(folder_path, "data.csv")
df.to_csv(file_path, index=False)
